In [ ]:
import pandas as pd
import numpy as np
from math import log, sqrt
from scipy.stats import norm
from scipy.optimize import minimize
import yfinance as yf

# Python Code: GARCH(1,1) on VIX (1-Day Ahead Forecast)

# -----------------------------
# Load VIX data
# -----------------------------
ticker = "^VIX"
start_date = "2015-01-01"

data = yf.download(ticker, start=start_date)
vix = data["Adj Close"].dropna()


# -----------------------------
# Use log returns of VIX
# -----------------------------
returns = np.log(vix / vix.shift(1)).dropna()
returns = returns.values


# -----------------------------
# GARCH(1,1) log-likelihood
# -----------------------------
def garch_log_likelihood(params, returns):
    omega, alpha, beta = params
    
    T = len(returns)
    sigma2 = np.zeros(T)
    sigma2[0] = np.var(returns)

    for t in range(1, T):
        sigma2[t] = omega + alpha * returns[t-1]**2 + beta * sigma2[t-1]

    loglik = -0.5 * np.sum(
        np.log(2 * np.pi) +
        np.log(sigma2) +
        returns**2 / sigma2
    )

    return -loglik  # minimize


# -----------------------------
# Parameter estimation
# -----------------------------
initial_params = np.array([0.000001, 0.05, 0.90])
bounds = [(1e-8, None), (0, 1), (0, 1)]

result = minimize(
    garch_log_likelihood,
    initial_params,
    args=(returns,),
    bounds=bounds
)

omega, alpha, beta = result.x


# -----------------------------
# One-day-ahead volatility forecast
# -----------------------------
last_return = returns[-1]
last_variance = np.var(returns)

forecast_variance = (
    omega +
    alpha * last_return**2 +
    beta * last_variance
)

forecast_volatility = sqrt(forecast_variance)


# -----------------------------
# Output
# -----------------------------
print("Estimated GARCH(1,1) parameters")
print(f"Omega: {omega:.6e}")
print(f"Alpha: {alpha:.4f}")
print(f"Beta : {beta:.4f}")
print(f"Alpha + Beta: {alpha + beta:.4f}")

print("\nOne-Day-Ahead Forecast")
print(f"Forecasted Daily Volatility: {forecast_volatility:.2%}")
print(f"Forecasted Annualized Volatility: {forecast_volatility * sqrt(252):.2%}")
